## Data preprocessing

In [ ]:
# Chargement des librairies nécessaires
library(ggplot2)
library(tidyverse)
library(gridExtra)
library(GGally)
library(plotly)
library(corrplot)
library(reshape2)
library(FactoMineR) 
library(factoextra)
library(glmnet) 
library(ggfortify)
library(pROC)
library(ROCR)
library(ggpubr)

In [ ]:
# Lecture des données
path <- "data/"
song <- read.csv(paste0(path, "spotify_songs.csv"), header = TRUE, sep = ",")

# Premières lignes du jeu de données
head(song)

# Vérification du contenu
summary(song)

In [ ]:
# Check the data types
str(song)

In [ ]:
# Drop the track_id, track_album_id, playlist_id columns
song <- song[, -c(1, 5, 9)]

# As factor the categorical variables track_artist, playlist_genre, playlist_subgenre, key, mode, playlist_name, track_album_name
song$playlist_name <- as.factor(song$playlist_name)
song$track_album_name <- as.factor(song$track_album_name)
song$track_artist <- as.factor(song$track_artist)
song$playlist_genre <- as.factor(song$playlist_genre)
song$playlist_subgenre <- as.factor(song$playlist_subgenre)
song$key <- factor(song$key, levels = c(-1, 0:11), labels = c("No key detected", "C", "C♯/D♭", "D", "D♯/E♭", "E", "F", "F♯/G♭", "G", "G♯/A♭", "A", "A♯/B♭", "B"))
song$mode <- factor(song$mode, levels = c(0, 1), labels = c("minor", "major"))

# track_album_release_date to date (if the full date is not available, we will use the first day of the year)
song$track_album_release_date <- as.Date(ifelse(nchar(song$track_album_release_date) != 10, 
                                                paste0(substr(song$track_album_release_date, 1, 4), "-01-01"), 
                                                song$track_album_release_date), 
                                         format = "%Y-%m-%d")

# Convert the duration_ms to seconds and rename the column to duration_s
song$duration_s <- song$duration_ms / 1000
song$duration_ms <- NULL

# Check the modified dataset
summary(song)
head(song)

In [ ]:
# Check for missing values
colSums(is.na(song))

# Drop the missing values
song <- na.omit(song)

# Check the modified dataset
colSums(is.na(song))

str(song)

## Dimensionnality reduction

### Principal Component Analysis

In [ ]:
# PCA analysis using FactoMineR
song_pca <- song[, c(3, 7, 9:10, 12, 14:20)]

# Perform PCA
pca <- PCA(song_pca,scale.unit = TRUE, graph = FALSE,ncp = 4,quali.sup = 2)

# Afficher le pourcentage de variance expliquée par chaque composante principale
fviz_eig(pca, addlabels = TRUE, ylim = c(0, 40))

# Calculer la variance cumulée
explained_variance <- pca$eig[, 2]  # La deuxième colonne contient le pourcentage de variance expliquée
cumulative_variance <- cumsum(explained_variance)

# Tracer la variance cumulée
plot(cumulative_variance, xlab = "Nombre de composantes principales", ylab = "Variance cumulée", type = "b")
abline(h = 80, col = "red", lty = 2)  # Ligne horizontale à 80% de variance expliquée

# Corrélation des variables
corrplot(pca$var$cor, is.corr = FALSE, method = "ellipse")

## Interprétation de l'Analyse en Composantes Principales (PCA)

### **Analyse des principales corrélations :**
- **PC1 (Composante Principale 1) :**  
  - **Corrélations négatives fortes :** `energy (-0.91)`, `loudness (-0.80)`.  
    → Cette composante sépare les morceaux énergiques et bruyants des morceaux plus doux.  
  - **Corrélation positive forte :** `acousticness (+0.72)`.  
    → Elle oppose également les morceaux acoustiques aux morceaux énergétiques.  

- **PC2 (Composante Principale 2) :**  
  - **Corrélations négatives :** `danceability (-0.68)`, `valence (-0.62)`.  
    → Cette composante distingue les morceaux dansants et joyeux des autres.  
  - **Corrélations positives :** `instrumentalness (+0.45)`, `duration_s (+0.38)`.  
    → Elle représente les morceaux plus longs et instrumentaux.  

- **PC3 (Composante Principale 3) :**  
  - **Corrélations positives :** `danceability (+0.46)`, `instrumentalness (+0.39)`, `duration_s (+0.45)`.  
    → Cette dimension semble être liée à la dansabilité et à la durée des morceaux.  
  - **Corrélation négative :** `tempo (-0.35)`.  
    → Elle oppose les morceaux rapides et lents.  

- **PC4 (Composante Principale 4) :**  
  - **Corrélation positive forte :** `speechiness (+0.71)`.  
    → Cette composante est dominée par la présence de paroles (rap).  
  - **Corrélation positive :** `liveness (+0.58)`.  
    → Elle met en avant les morceaux enregistrés en live.  

### **Interprétation globale :**
- **PC1 sépare les morceaux énergiques et bruyants des morceaux acoustiques.**  
- **PC2 distingue les morceaux dansants et joyeux des morceaux instrumentaux et longs.**  
- **PC3 différencie les morceaux en fonction de leur dansabilité et de leur tempo.**  
- **PC4 met en avant les morceaux avec beaucoup de paroles et ceux enregistrés en live.**  

In [ ]:
# Tracer les variables de l'ACP
fviz_pca_var(pca)

# Tracer les individus de l'ACP avec la contribution colorée
fviz_pca_ind(pca, col.ind = "contrib", label = "none", gradient.cols = c("white", "#2E9FDF", "#FC4E07"))

# Tracer les individus de l'ACP avec la contribution colorée
fviz_pca_ind(pca, col.ind = song$playlist_genre, label = "none", addEllipses = TRUE, ellipse.level = 0.95, alpha.ind = 0.1, axes = c(1, 3))